# Test of OpenCV + FastAI for video capture

<br>
<br>

### For reference:
<img src='https://i.pinimg.com/originals/fa/bb/70/fabb7087b0cffe30530e7df9ec1d0b88.png' width="400" height="500"></img>
<br>
<br>

In [1]:
# !pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()

# RUN_NAME = '20210116-2147 - arch=xresnet50 - samples=1000 frozen=3 epochs=3 bs=20 res=300 _data=external preuntrained percent=88'
# RUN_NAME = '20210116-2108 - arch=xresnet18 - samples=480 frozen=2 epochs=3 bs=8 res=300 _data=frank_pretrained=false_percent=71'

# RUN_NAME = '20210118-2100 - arch=xresnet50 - samples=1000 frozen=0 epochs=25 bs=20 res=300 _data=external'
# RUN_NAME = '200927-0730-RESNET101-384px-FALL-ft3f4'
RUN_NAME = '20210119-0130 - arch=xresnet50 - samples=1000 frozen=0 epochs=25 bs=20 res=300 _data=external'
# RUN_NAME = '20210119-2152 - arch=xresnet50_deep - samples=1000 frozen=0 epochs=25 bs=20 res=300 _data=external'
# RUN_NAME = '20210118-2000 - arch=xresnet50 - samples=1000 frozen=0 epochs=25 bs=20 res=300 _data=external'
# RUN_NAME = '20210119-0130 - arch=xresnet50 - samples=1000 frozen=0 epochs=25 bs=20 res=300 _data=external'




SAVE_IMAGES_TO_FILE = False  # flag if we want to capture the image to disk

SYMBOL = None

RESOLUTION = 300


# from fastbook import *
from fastbook import *
from fastai.vision.all import *


from fastai.vision.widgets import *
import fastai.vision
import fastai
from fastai.learner import *
import fastprogress

import numpy as np
import cv2 as cv

import os




<br>
<br>

## Load the model

In [2]:
learn_inf = load_learner(f'../models/{RUN_NAME}.pkl', cpu=True)

# load_model(f'./models/{RUN_NAME}_10.pth', learn_inf, with_opt=True, opt=Adam)

In [3]:
# import random

# # Needed to pass into the DataBlock    
# def get_fnames(path): 
#     retlist = get_image_files('../data/frank/Training_Set')
#     return random.sample(retlist, len(retlist))
        

    
    
    
# signs = DataBlock(
#     blocks=(ImageBlock, CategoryBlock), 
#     get_items=get_fnames, 
#     splitter=RandomSplitter(valid_pct=0.3, seed=42),
#     get_y=parent_label,
#     item_tfms=CropPad(RESOLUTION, pad_mode='zeros')
#    ,    batch_tfms=aug_transforms(do_flip=True, size=RESOLUTION, batch=False, max_zoom=1.0, mult=1, pad_mode='zeros'))


In [4]:
# funky_path = f'../data/frank/Training_Set'

# dls = signs.dataloaders(funky_path, bs=32)

In [5]:
# learn_inf.dls = dls

<br>
<br>

## Function definitions

In [6]:
def setup_camera() -> cv.VideoCapture:
    '''
    Set up the camera source
    '''
#     cap = cv.VideoCapture(0)
    # cap = cv.VideoCapture('http://127.0.0.1:4747/video')
#     cap = cv.VideoCapture('http://10.0.0.67:4747/video')  # S7
    cap = cv.VideoCapture('http://10.0.0.74:4747/video')  # N10

    # cap = cv.VideoCapture('Droidcam')
#     cap = cv.VideoCapture('http://10.0.0.144:4747/video')  # N8
    
    print(f'Autofocus status: {cap.set(cv.CAP_PROP_AUTOFOCUS, 0)}')
    print(f'Manual focus to shortest distance: {cap.set(cv.CAP_PROP_FOCUS, 0)}')

    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error opening video stream or file!")
    
    return cap




def close_camera():
    '''
    Close all capture devices and destroy open capture windows
    '''
    cap.release()
    cv.destroyAllWindows()
    
    


def print_stats():
    '''
    Print some basic stats to stdout
    '''
    
    print(f'Frame shape = {frame.shape}')
    print(f'Total number of Frames = {nframe}')
    print(f'Number of frames processed = {n_proc_frames // PROC_NTH_FRAME}')
    
    
    
# @TODO: Factor out the directory creation logic - slow/redundant    
def save_images(frame, prframe):
    '''
    write small and large images to jpeg
    '''
    symbol_dir = SYMBOL
    
    #seperate the directory path
    datadir_small = f'../data/{maindir}-S/{symbol_dir}'
    datadir_large = f'../data/{maindir}-L/{symbol_dir}'

    # build the entire directory structure if it doesn't exist
    if not os.path.isdir(datadir_small):
        os.makedirs(datadir_small)
        
    if not os.path.isdir(datadir_large):
        os.makedirs(datadir_large)
    
    # create a directory+filename template
    ftemplate_small = f'{datadir_small}/{file_prefix}-{n_save_frames}.jpg'
    ftemplate_large = f'{datadir_large}/{file_prefix}-{n_save_frames}.jpg'
    
    # write the image to a file
#     cv.imwrite(ftemplate_small, prframe)
    cv.imwrite(ftemplate_large, frame)
    
    frame = cv.putText(frame, f'Saved Frame {SYMBOL} #{n_save_frames}', 
                   org=(20,40), fontFace=cv.FONT_HERSHEY_PLAIN, 
                   fontScale=2, color=(0,255,255), thickness=2,
                   lineType=cv.LINE_AA) 
    
    # Display the frame with the writing on it
    cv.imshow('What the Camera Sees:',frame)
    
    
    
# def disable_progress():
#     fastprogress.fastprogress.NO_BAR = True
#     master_bar, progress_bar = fastprogress.fastprogress.force_console_behavior()
# #     fastai.basic_train.master_bar, fastai.basic_train.progress_bar = master_bar, progress_bar
    
# # def enable_progress():
# #     fastai.basic_train.master_bar, fastai.basic_train.progress_bar = fastprogress.master_bar, fastprogress.progress_bar
    


<br>
<br>

## Open the video camera, loop for every Frame, massage the image and make a prediction.  

#### This is the main code cel.  It has 2 purposes. When you hit the spacebar, it will either:
1. Make an ASL alphabet translation or 
2. If SAVE_IMAGES_TO_FILE is True, it will save an image to the data dir 

In [7]:
from fastai.vision  import *
import torch
# import PIL

PROC_NTH_FRAME = 8  # Skip every N-1 frames - increase this if your computer lags

maindir = 'frank-ledlights'  # these are image saving parameters dir/file
file_prefix = 'oldwebcam_leds'

n_proc_frames, nframe = 0,0  # number of frames
n_save_frames = 0  # number of captured frames for saving to file

# declare here to widen scope
pred, probs, pred_idx = [0], [0], 0
frame, prframe = None, None

if SAVE_IMAGES_TO_FILE:
    SYMBOL = input('Enter Symbol of interest: ')[0]
    
cap = setup_camera()


# "Game" Loop ... for every frame
while(cap.isOpened()):
    nframe += 1
    ret, frame = cap.read()  # Capture frame
    wait_ret = cv.waitKey(1)  # key char value if any

    # Break out of the loop if the user presses 'Q'
    if wait_ret & 0xFF == ord('q'):
        print_stats()
        break
    
    # only process the rest if the capture was successful
    if not ret:
        print("Can't read frame from capture source!")
        break  # break out of loop to let the camera close

    # crop the original 640x480 image down to a centered square 480x480
#     frame = frame[:, 80:-80]

    # mirror the image horizontally
    frame = cv.flip(frame, 1)


    # process only if the space key is hit and
    # only process every PROC_NTH_FRAME frames 
    if (nframe % PROC_NTH_FRAME == 0):  # and wait_ret & 0xFF == ord(' ')  
        n_proc_frames +=1  # num processed frames


        # this version for resnet34 w/ 300px image
        prframe = cv.resize(frame[:, 80:-80], (300, 300))
#         prframe = frame[:,80:-80]

        
        # Display the frame that we pass through the predictor
        cv.imshow('What the Predictor Sees:', prframe)

        if SAVE_IMAGES_TO_FILE :
            if wait_ret & 0xFF == ord(' '):  # only save when pressing the spacebar
                n_save_frames = n_save_frames + 1
                save_images(frame, prframe)
            continue  # no need to predict

        # create the prediction
        with learn_inf.no_bar():
            pred,pred_idx,probs = learn_inf.predict(prframe)

            
            
        # write the prediction as text on the frame
        if probs[pred_idx] > 0.5:
            frame = cv.putText(frame, f'Predict: {pred}, Conf: {probs[pred_idx]:.02f}', 
                               org=(20,40), fontFace=cv.FONT_HERSHEY_PLAIN, 
                               fontScale=2, color=(0,255,255), thickness=2,
                               lineType=cv.LINE_AA) 

        # Display the original frame
        cv.imshow('What the Camera Sees:',frame)

# Release the capture object
close_camera()


Autofocus status: False
Manual focus to shortest distance: False
Frame shape = (480, 640, 3)
Total number of Frames = 668
Number of frames processed = 10


In [8]:
close_camera()                

<br>
<br>
<br>
<br>
<br>


# The rest of this file is for reference only.  

<br>
<br>
<br>



<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>


# Snippets

```python
# rotate the image
frame = cv.rotate(frame, cv.ROTATE_90_COUNTERCLOCKWISE)
```

In [9]:
# learn.export(f'../models/{RUN_NAME}.pkl')
# path = Path('../models')
# path.ls(file_exts='.pkl')